## Trabalho 1 - Reconhecimento de Padrões
Aluno: Luis Felipe Carneiro de Souza

Matrícula: 535049

> Os códigos deste trabalho foram desenvolvido com bases no material em MATLAB/OCTAVE disponibilizado pelo professor

importação de bibliotecas utilizadas

In [1]:
import pandas as pd
import numpy as np
import time
from scipy.spatial.distance import minkowski
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns

Lendo o conjunto de dados como um DataFrame

In [2]:
conjunto = pd.read_csv('column_3C.dat', header=None, sep='\s+')
#print(conjunto.info)
conjunto = conjunto.sample(frac=1)
conjunto = conjunto.iloc[:200, :] # Pegando os primeiros 60 elementos para os códigos executarem mais rápido
conjunto.head()

,0,1,2,3,4,5,6
187,60.42,5.27,59.81,55.15,109.03,30.27,SL
53,30.15,11.92,34.00,18.23,112.68,11.46,DH
288,74.57,15.72,58.62,58.84,105.42,0.60,NO
55,47.66,13.28,36.68,34.38,98.25,6.27,DH
196,63.77,12.76,65.36,51.01,89.82,56.00,SL


Separando o conjunto de dados entre variáveis de entrada e variáveis resposta

In [3]:
x = conjunto.iloc[:, 0:-1]
y = conjunto.iloc[:, -1]

In [4]:
x = conjunto.iloc[:, 0:-1]
y = conjunto.iloc[:, -1]

x_n = normalize(x, norm='l2', axis=1)
x = pd.DataFrame(x_n, index=x.index, columns=x.columns)

Dicionário de proporção de separação entre conjunto de treino e teste

In [5]:
sep = {'20/80': {'train': 0.2, 'test': 0.8},
       '30/70': {'train': 0.3, 'test': 0.7},
       '50/50': {'train': 0.5, 'test': 0.5},
       '70/30': {'train': 0.7, 'test': 0.3},
       '80/20': {'train': 0.8, 'test': 0.2}}

### Classificador Vizinho Mais Próximo (distância de Minkowski de ordens m ∈ {0,5; 2/3; 1; 3/2; 2; 5/2})

In [ ]:
resultados_knn = {}
for s in sep.keys():
    resultados_knn[s] = {}

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    for m in [0.5, 2/3, 1, 3/2, 2, 5/2]: # diferentes ordens da distância de Minkowski

        print(f'\n\nSeparação {s} e distância de Minkowski de ordem {m}\n')

        Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
        inicio = time.perf_counter()

        for rep in range(100): # 100 rodadas de treino
            #print(f'Rodada de número {rep + 1}')

            x = x.sample(frac=1) # embaralha as amostras
            y = y.loc[x.index] # embaralha as classes

            # conjunto de treino
            x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
            
            # conjunto de teste
            x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

            # loop que itera sobre o conjunto de teste
            acerto = 0 # contador de acertos
            for i in range(len(x_tst)):
                x_new = np.array(x_tst.iloc[i, :]) # nova amostra a ser classificada
                y_new = y_tst.iloc[i] # classe da nova amostra

                dist_array = []
                for j in range(len(x_trn)):
                    dist_array.append(minkowski(x_new, np.array(x_trn.iloc[j, :]), p=m)) # distância de Minkowski de ordem m
                
                j_min = np.argmin(dist_array) # índice do menor valor de distância

                if y_trn.iloc[j_min] == y_new: # verifica se a classificação foi correta
                    acerto += 1
        
            Pacerto.append(acerto / len(x_tst)) # calcula a porcentagem de acertos
        fim = time.perf_counter()

        resultados_knn[s][m] = {'media': np.mean(Pacerto),
                           'desvio': np.std(Pacerto),
                           'mediana': np.median(Pacerto),
                           'minimo': np.min(Pacerto),
                           'maximo': np.max(Pacerto),
                           'tempo': fim - inicio}
resultados_knn

### Classificador Distância Mínima ao Centróide

In [9]:
resultados_centroide = {}
C = conjunto.iloc[:, -1].unique()
for s in sep.keys():

    #print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    acc_por_classe_lists = {classe: [] for classe in C}

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        x = x.sample(frac=1)
        y = y.loc[x.index]
        
        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.mean()

        class_hits = {classe: 0 for classe in C}
        class_totals = {classe: 0  for classe in C}

        acerto = 0 # contador de acertos
        for i in range(len(x_tst)): # loop que itera sobre o conjunto de teste
            x_new = np.array(x_tst.iloc[i, :]) # nova amostra a ser classificada
            y_true = y_tst.iloc[i] # classe da nova amostra

            dist_array = []
            for k in C:
                #print(x_new, np.array(centroides[k]))
                dist_array.append(minkowski(x_new, np.array(centroides[k]), p=2)) # distância Euclidiana ao centróide da classe k
            
            k_min = np.argmin(dist_array) # índice do menor valor de distância
            y_pred = C[k_min]

            class_totals[y_true] += 1

            if y_pred == y_true:
                acerto += 1
                class_hits[y_true] += 1
        
        Pacerto.append(acerto / len(x_tst))

        for classe in C:
            total = class_totals[classe]
            hits = class_hits[classe]

            if total > 0:
                acc_da_classe_nesta_rodada = hits / total
            else:
                acc_da_classe_nesta_rodada = np.nan

            acc_por_classe_lists[classe].append(acc_da_classe_nesta_rodada)

    fim = time.perf_counter()

    stats_por_classe = {}
    for classe, lista_de_acuracias in acc_por_classe_lists.items():
        stats_por_classe[classe]= {
            'media': np.nanmean(lista_de_acuracias),
            'desvio': np.nanstd(lista_de_acuracias),
            'mediana': np.nanmedian(lista_de_acuracias),
            'minimo': np.nanmin(lista_de_acuracias),
            'maximo': np.nanmax(lista_de_acuracias),
        }

    resultados_centroide[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'stats_por_classe': stats_por_classe,
                    'tempo': fim - inicio}
resultados_centroide

{'20/80': {'media': np.float64(0.7384999999999999),
  'desvio': np.float64(0.03330399750780677),
  'mediana': np.float64(0.7375),
  'minimo': np.float64(0.66875),
  'maximo': np.float64(0.8125),
  'stats_por_classe': {'SL': {'media': np.float64(0.9217168519194702),
    'desvio': np.float64(0.03381659344167383),
    'mediana': np.float64(0.9315068493150684),
    'minimo': np.float64(0.8133333333333334),
    'maximo': np.float64(0.9868421052631579)},
   'DH': {'media': np.float64(0.5041727859970311),
    'desvio': np.float64(0.12368369346816654),
    'mediana': np.float64(0.4827586206896552),
    'minimo': np.float64(0.22857142857142856),
    'maximo': np.float64(0.8275862068965517)},
   'NO': {'media': np.float64(0.6179436632556008),
    'desvio': np.float64(0.1059036987556251),
    'mediana': np.float64(0.6273148148148149),
    'minimo': np.float64(0.4),
    'maximo': np.float64(0.8490566037735849)}},
  'tempo': 3.200228000059724},
 '30/70': {'media': np.float64(0.7373571428571429),
  

In [ ]:
C = np.sort(conjunto.iloc[:, -1].unique())
C

### Classificador Distância Mínima ao Centróide (versão robusta a outliers)

In [ ]:
resultados_centroide_robusto = {}
C = conjunto.iloc[:, -1].unique()
for s in sep.keys():

    #print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        x = x.sample(frac=1)
        y = y.loc[x.index]
        
        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.median()

        acerto = 0 # contador de acertos
        for i in range(len(x_tst)):
            x_new = np.array(x_tst.iloc[i, :]) # nova amostra a ser classificada
            y_new = y_tst.iloc[i] # classe da nova amostra

            dist_array = []
            for k in C:
                #print(x_new, np.array(centroides[k]))
                dist_array.append(minkowski(x_new, np.array(centroides[k]), p=1)) # distância quarteirao ao centróide da classe k
            
            k_min = np.argmin(dist_array) # índice do menor valor de distância

            if C[k_min] == y_new:
                acerto += 1 # verifica se a classificação foi correta
        
        Pacerto.append(acerto / len(x_tst))

    fim = time.perf_counter()

    resultados_centroide_robusto[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'tempo': fim - inicio}
resultados_centroide_robusto

### Classificador de Máxima Correlação

In [ ]:
resultados_similaridade = {}
C = conjunto.iloc[:, -1].unique()

x = conjunto.iloc[:, 0:-1]
y = conjunto.iloc[:, -1]

x_n = normalize(x, norm='l2', axis=1)
x = pd.DataFrame(x_n, index=x.index, columns=x.columns)

for s in sep.keys():

    #print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        x = x.sample(frac=1)
        y = y.loc[x.index]
        
        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.mean()

        acerto = 0 # contador de acertos
        for i in x_tst.index:
            x_new = np.array(x_tst.loc[i, :]) # nova amostra a ser classificada
            y_new = y_tst.loc[i] # classe da nova amostra

            sim = []
            for k in C:
                sim.append(np.dot(x_new, np.array(centroides[k]))) # distância Euclidiana ao centróide da classe k
            
            k_min = np.argmax(sim) # índice do menor valor de distância

            if C[k_min] == y_new:
                acerto += 1 # verifica se a classificação foi correta
        
        Pacerto.append(acerto / len(x_tst))

    fim = time.perf_counter()

    resultados_similaridade[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'tempo': fim - inicio}
resultados_similaridade

### Resultados

In [ ]:
# 1. Consolidação dos Dados em um único DataFrame

# Lista para armazenar todos os resultados
dados_grafico = []

# Processa os resultados do k-NN
for separacao, resultados_m in resultados_knn.items():
    for m, metricas in resultados_m.items():
        dados_grafico.append({
            'classificador': 'k-NN',
            'separacao': separacao,
            'proporcao_treino': int(separacao.split('/')[0]),
            'parametro_m': f'{m:.2f}', # Formata o 'm' para ter 2 casas decimais
            **metricas # Adiciona todas as outras métricas (media, desvio, etc.)
        })

# Processa os resultados do Centróide
for separacao, metricas in resultados_centroide.items():
    dados_grafico.append({
        'classificador': 'Centróide',
        'separacao': separacao,
        'proporcao_treino': int(separacao.split('/')[0]),
        'parametro_m': 'N/A',
        **metricas
    })

# Processa os resultados do Centróide Robusto
for separacao, metricas in resultados_centroide_robusto.items():
    dados_grafico.append({
        'classificador': 'Centróide Robusto',
        'separacao': separacao,
        'proporcao_treino': int(separacao.split('/')[0]),
        'parametro_m': 'N/A',
        **metricas
    })

# Processa os resultados de Máxima Similaridade
for separacao, metricas in resultados_similaridade.items():
    dados_grafico.append({
        'classificador': 'Máx. Similaridade',
        'separacao': separacao,
        'proporcao_treino': int(separacao.split('/')[0]),
        'parametro_m': 'N/A',
        **metricas
    })

# Cria o DataFrame final
df_final = pd.DataFrame(dados_grafico)


In [ ]:
# 2. Geração do Gráfico de Barras Completo

# Ordena o DataFrame para o gráfico seguir a ordem dos experimentos
ordem_classificador = ['k-NN', 'Centróide', 'Centróide Robusto', 'Máx. Similaridade']
df_final['classificador'] = pd.Categorical(df_final['classificador'], categories=ordem_classificador, ordered=True)
df_ordenado = df_final.sort_values(['classificador', 'proporcao_treino', 'parametro_m'])

# Cria um rótulo único para cada barra para facilitar a visualização no gráfico
df_ordenado['rotulo'] = df_ordenado['classificador'].astype(str) + ' | ' + \
                        df_ordenado['separacao'].astype(str) + ' | m=' + \
                        df_ordenado['parametro_m'].astype(str)

# Ajusta rótulos para classificadores que não usam o parâmetro 'm'
df_ordenado['rotulo'] = df_ordenado['rotulo'].str.replace('\| m=N/A', '', regex=True)

# Configura o estilo do gráfico
sns.set_theme(style="whitegrid", palette="viridis")

# Plota o gráfico
plt.figure(figsize=(20, 10))
ax = sns.barplot(data=df_ordenado, x='rotulo', y='media')

# Adiciona as barras de erro (desvio padrão)
plt.errorbar(x=df_ordenado['rotulo'], y=df_ordenado['media'],
             yerr=df_ordenado['desvio'], fmt='none', c='black', capsize=3)

plt.title('Comparação Completa da Acurácia Média de Todos os Classificadores e Variações', fontsize=20)
plt.xlabel('Classificador | Separação Treino/Teste | Parâmetro (m)', fontsize=14)
plt.ylabel('Acurácia Média', fontsize=14)

# Rotaciona os rótulos do eixo X para que não se sobreponham
plt.xticks(rotation=90)

# Ajusta o limite do eixo Y para dar mais destaque às diferenças de performance
min_media = df_ordenado['media'].min()
max_media = df_ordenado['media'].max()
#plt.ylim(min_media * 0.9, max_media * 1.05)

plt.tight_layout() # Garante que todos os elementos do gráfico fiquem visíveis
plt.show()

In [ ]:
# Gráfico de Barras para o Tempo de Execução

# Configura o estilo do gráfico
sns.set_theme(style="whitegrid", palette="plasma")

# Plota o gráfico
plt.figure(figsize=(20, 10))
sns.barplot(data=df_ordenado, x='rotulo', y='tempo')

plt.title('Comparação do Tempo de Execução (em segundos)', fontsize=20)
plt.xlabel('Classificador | Separação Treino/Teste | Parâmetro (m)', fontsize=14)
plt.ylabel('Tempo Total para 100 Rodadas (s)', fontsize=14)

# Rotaciona os rótulos do eixo X para que não se sobreponham
plt.xticks(rotation=90)

plt.tight_layout() # Garante que todos os elementos do gráfico fiquem visíveis
plt.show()